In [ ]:
# select 1000 random cards from datasets/images and save them to a new directory
import os
import random

# Create a new directory to save the selected cards
output_dir = "datasets/testing_cards"
os.makedirs(output_dir, exist_ok=True)

# Get a list of all image files in the datasets/images directory
image_dir = "datasets/cards"
all_images = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

# Randomly select 1000 images
selected_images = random.sample(all_images, 1000)

# Copy the selected images to the new directory
for i in range(1000):
    src_path = os.path.join(image_dir, selected_images[i])
    dst_path = os.path.join(output_dir, str(i) + ".jpg")
    with open(src_path, 'rb') as src_file:
        with open(dst_path, 'wb') as dst_file:
            dst_file.write(src_file.read())

In [21]:
import pandas as pd
from pathlib import Path
import glob
import random
import os
import requests
from tqdm import tqdm  # for progress bar

# Set your path
path = r'C:\Users\tingw\Downloads\unsplash-research-dataset-lite-latest'

# Find the photos file (handles .tsv or .tsv.gz)
photo_files = glob.glob(str(Path(path) / "photos.tsv*"))

if not photo_files:
    raise FileNotFoundError(f"No photos.tsv file found in {path}")

# Load the photos data
photos_df = pd.read_csv(photo_files[0], sep='\t')

# Get 100 random samples
random_samples = photos_df.sample(n=100, random_state=42)  # random_state for reproducibility

# Display the result
print(f"Loaded {len(random_samples)} random photos:")
print(random_samples[['photo_id', 'photo_image_url']].head())  # Show key columns

# Create output directory
output_dir = "datasets/background"
os.makedirs(output_dir, exist_ok=True)

# Download function with error handling
def download_image(url, filepath):
    try:
        response = requests.get(url, stream=True, timeout=10)
        response.raise_for_status()
        with open(filepath, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        return True
    except Exception as e:
        print(f"Failed to download {url}: {e}")
        return False

# Download the images with progress bar
success_count = 0
for i in range(100):
    row = random_samples.iloc[i]
    image_url = row['photo_image_url']  # Use the URL column
    dst_path = os.path.join(output_dir, f"{i}.jpg")
    if download_image(image_url, dst_path):
        success_count += 1

print(f"Successfully downloaded {success_count}/{len(random_samples)} images to {output_dir}")

Loaded 100 random photos:
          photo_id                                    photo_image_url
6868   6Qo7dWR2wZM  https://images.unsplash.com/photo-158205618674...
24016  1DPIP44atys  https://images.unsplash.com/photo-141919321743...
9668   P_pjzMeIbj8  https://images.unsplash.com/photo-1558966151-c...
13640  Y1H182HRPEw  https://images.unsplash.com/photo-1544989387-1...
14018  tnEdvYk3IOw  https://images.unsplash.com/photo-144708252926...
Successfully downloaded 100/100 images to datasets/background


In [ ]:
import cv2
import numpy as np
import os
from tqdm import tqdm

# Create output directory
os.makedirs("images/train", exist_ok=True)

# Get list of available files
backgrounds = [f"datasets/background/{f}" for f in os.listdir("datasets/background") if f.endswith((".jpg", ".png"))]
cards = [f"datasets/testing_cards/{f}" for f in os.listdir("datasets/testing_cards") if f.endswith((".jpg", ".png"))]

print(f"Found {len(backgrounds)} backgrounds and {len(cards)} cards")

def apply_transformations(image, max_perspective=20, max_rotation=30):
    h, w = image.shape[:2]
    
    # Add padding to prevent edge clipping
    pad_size = int(max(h, w) * 0.3)
    image = cv2.copyMakeBorder(image, pad_size, pad_size, pad_size, pad_size, 
                             cv2.BORDER_CONSTANT, value=(0,0,0,0))
    
    # Apply perspective warp
    if max_perspective > 0:
        src_points = np.float32([[pad_size, pad_size], 
                               [w+pad_size, pad_size], 
                               [w+pad_size, h+pad_size], 
                               [pad_size, h+pad_size]])
        
        max_offset_x = int(w * max_perspective/100)
        max_offset_y = int(h * max_perspective/100)
        
        dst_points = np.float32([
            [pad_size + np.random.randint(-max_offset_x, max_offset_x), 
             pad_size + np.random.randint(-max_offset_y, max_offset_y)],
            [w+pad_size - np.random.randint(-max_offset_x, max_offset_x), 
             pad_size + np.random.randint(-max_offset_y, max_offset_y)],
            [w+pad_size - np.random.randint(-max_offset_x, max_offset_x), 
             h+pad_size - np.random.randint(-max_offset_y, max_offset_y)],
            [pad_size + np.random.randint(-max_offset_x, max_offset_x), 
             h+pad_size - np.random.randint(-max_offset_y, max_offset_y)]
        ])
        
        M = cv2.getPerspectiveTransform(src_points, dst_points)
        image = cv2.warpPerspective(image, M, (w+2*pad_size, h+2*pad_size), 
                                  borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0,0))
    
    # Apply rotation
    if max_rotation > 0:
        angle = np.random.uniform(-max_rotation, max_rotation)
        M = cv2.getRotationMatrix2D((image.shape[1]//2, image.shape[0]//2), angle, 1.0)
        image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]), 
                             flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, 
                             borderValue=(0,0,0,0))
    
    # Find non-zero area and crop
    if image.shape[2] == 4:
        gray = image[:,:,3]
    else:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        cnt = max(contours, key=cv2.contourArea)
        x,y,w,h = cv2.boundingRect(cnt)
        image = image[y:y+h, x:x+w]
    
    return image

for i in tqdm(range(10)):  # Generate 10 sample images
    # Load background
    bg_path = backgrounds[np.random.randint(0, len(backgrounds))]
    bg = cv2.imread(bg_path)
    if bg is None:
        print(f"Failed to load background: {bg_path}")
        continue
    
    # Resize background to minimum 1500x1200 to accommodate transformed cards
    bg = cv2.resize(bg, (max(1500, bg.shape[1]), max(1200, bg.shape[0])))
    
    for _ in range(np.random.randint(2,5)): 
        card_path = cards[np.random.randint(0, len(cards))]
        card_img = cv2.imread(card_path, cv2.IMREAD_UNCHANGED)
        if card_img is None:
            print(f"Failed to load card: {card_path}")
            continue
        
        # Increase card size (1.0 to 2.0x)
        scale_factor = np.random.uniform(1.0, 2.0)
        card_img = cv2.resize(card_img, None, fx=scale_factor, fy=scale_factor, 
                            interpolation=cv2.INTER_LANCZOS4)
        
        # Create alpha channel if doesn't exist
        if card_img.shape[2] == 3:
            card_img = cv2.cvtColor(card_img, cv2.COLOR_BGR2BGRA)
            card_img[:, :, 3] = 255
        
        # Apply transformations with edge preservation
        card_img = apply_transformations(card_img, 
                                       max_perspective=np.random.uniform(10,20), 
                                       max_rotation=np.random.uniform(0,30))
        
        # Skip if transformation failed
        if card_img.size == 0:
            continue
            
        # Random position with safe margins
        margin_x = int(card_img.shape[1] * 0.3)
        margin_y = int(card_img.shape[0] * 0.3)
        max_x = bg.shape[1] - card_img.shape[1] - margin_x
        max_y = bg.shape[0] - card_img.shape[0] - margin_y
        
        if max_x <= margin_x or max_y <= margin_y:
            continue
            
        x = np.random.randint(margin_x, max_x)
        y = np.random.randint(margin_y, max_y)
        
        # Composite using alpha channel
        if card_img.shape[2] == 4:
            alpha = card_img[:, :, 3][..., np.newaxis]/255.0
            roi = bg[y:y+card_img.shape[0], x:x+card_img.shape[1]]
            bg[y:y+card_img.shape[0], x:x+card_img.shape[1]] = (roi * (1 - alpha) + card_img[:, :, :3] * alpha).astype(np.uint8)
        else:
            bg[y:y+card_img.shape[0], x:x+card_img.shape[1]] = card_img
    
    # Add slight vignette effect
    rows, cols = bg.shape[:2]
    kernel = cv2.getGaussianKernel(cols, cols/3) * cv2.getGaussianKernel(rows, rows/3).T
    mask = kernel / kernel.max()
    bg = (bg * (mask[..., np.newaxis] * 0.8 + 0.2)).astype(np.uint8)
    
    cv2.imwrite(f"images/train/{i}.jpg", bg)

print("Synthetic dataset generation complete!")

Found 100 backgrounds and 1000 cards


  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (3032,4048,3) (4048,3032,1) 

: 

In [ ]:
cards_df = pd.read_csv('cardsRaw.csv')

attacks_data = []
cardmarket_data = []
small_images_data = []
large_images_data = []
resistances_data = []
# legalities_data is already covered in set data
for idx, row in cards_df.iterrows():




,abilities,artist,ancientTrait,attacks,cardmarket,convertedRetreatCost,evolvesFrom,flavorText,hp,id,...,rarity,resistances,retreatCost,rules,set,subtypes,supertype,tcgplayer,types,weaknesses
0,NaN,Kagemaru Himeno,NaN,"[{'name': 'Second Strike', 'cost': ['Metal', '...",{'url': 'https://prices.pokemontcg.io/cardmark...,4.0,Lairon,You can tell its age by the length of its iron...,140.0,hgss4-1,...,Rare Holo,"[{'type': 'Psychic', 'value': '-20'}]","['Colorless', 'Colorless', 'Colorless', 'Color...",NaN,"{'id': 'hgss4', 'images': {'symbol': 'https://...",['Stage 2'],Pokémon,{'url': 'https://prices.pokemontcg.io/tcgplaye...,['Metal'],"[{'type': 'Fire', 'value': '×2'}]"
